# 3.3.1 DG for the acoustic wave equation
We consider the acoustic problem
\begin{align} 
      \partial_{t} p  +  c \operatorname{div}(\mathbf{u}) &= 0 \quad \text{ in } \Omega \times I, \\
      \partial_{t} \mathbf{u}  +  c \nabla p &= 0 \quad \text{ in } \Omega \times I, \\
%      p (0,\cdot) &= p(1,\cdot), \quad
%      p (\cdot,0) = p(\cdot,1), \label{eq:per2b}\\
%      \mathbf{q} (0,\cdot) &= \mathbf{q}(1,\cdot), \quad
%      \mathbf{q} (\cdot,0) = \mathbf{q}(\cdot,1), \label{eq:per2bb}\\
      p &= p_0 \!\!\! \quad \text{ on } \Omega \times \{0\},\\
      u &= 0 \quad \text{ on } \Omega \times \{0\}.
\end{align}
Here $p$ is the acoustic pressure (the local deviation from the ambient pressure) and $\mathbf{u}$ is the local velocity.

A simple grid:

In [ ]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.1))
Draw(mesh)

Find $p: [0,T] \to \bigoplus_{T\in\mathcal{T}_h} \mathcal{P}^{k+1}(T)$ and $\mathbf{u}: [0,T] \to \bigoplus_{T\in\mathcal{T}_h} [\mathcal{P}^k(T)]^d$ so that 


\begin{align*}
    (\partial_t \mathbf{u}, v) &= b_h(p,v) & &&& \forall v,\\
    (\partial_t p, q) &= & - b_h(q,\mathbf{u}) &&& \forall q\\    
\end{align*}

with the centered flux approximation:
$$
  b_h(p,v) = \sum_{T} \int_T \nabla p \cdot v + \int_{\partial T} ( \hat{p} - p) v_n 
$$

Here $\hat{p}$ is the centered approximation i.e. $\hat{p} = \{\!\!\{p\}\!\!\}$.


In [ ]:
order = 4
fes_p = L2(mesh, order=order+1, all_dofs_together=True, order_policy=ORDER_POLICY.CONSTANT)
fes_u = VectorL2(mesh, order=order, piola=True, all_dofs_together=True, order_policy=ORDER_POLICY.CONSTANT)
fes = FESpace( [fes_p,fes_u] )
gfu = GridFunction(fes)
Draw(gfu.components[1], mesh, "u")
Draw(gfu.components[0], mesh, "p", min=-0.1, max=0.1, autoscale=False, sd=3) 

#### What is the flag `piola` doing?

The `VectorL2` space uses the following definition of basis functions on the mesh. 

Let $\hat{\varphi}(\hat{x})$ be a (vectorial) basis function on the reference element $\hat{T}$, $\Phi: \hat{T} \to T$ with $x = \Phi(\hat{x})$ and $F= D\Phi$, then

$$
  \varphi(x) = \left\{ \begin{array}{rlll} 
  \hat{\varphi}(\hat{x}), & \texttt{piola=False}, & \texttt{covariant=False}, & \text{(default)} \\
  \frac{1}{|\operatorname{det} F|} \cdot F \cdot \hat{\varphi}(\hat{x}), & \texttt{piola=True},  & \texttt{covariant=False}, \\
  F^{-T} \cdot \hat{\varphi}(\hat{x}), & \texttt{piola=False}, & \texttt{covariant=True}. \\
  \end{array}
  \right.
$$

If you want to inspect the basis functions for the three different cases take a look at [l2trafo.ipynb](l2trafo.ipynb)

### Inverse mass matrix operations:
Combining `Embedding` and the inverse mass matrix operation for `fes_p` allows to realize the following block inverse operations acting on `fes`:

$$
\underbrace{\left( \begin{array}{cc} M_{p}^{-1} & 0 \\ 0 & 0 \end{array} \right)}_{\texttt{invp}} = 
\underbrace{\left( \begin{array}{c} I \\ 0 \end{array} \right)}_{\texttt{emb_p}} \cdot \underbrace{M_{u}^{-1}}_{\texttt{invmassp}} \cdot \underbrace{\left( \begin{array}{cc} I & 0 \end{array} \right)}_{\texttt{emb_p.T}}
$$

In [ ]:
pdofs = fes.Range(0);
emb_p = Embedding(fes.ndof, pdofs)
invmassp = fes_p.Mass(1).Inverse()
invp = emb_p @ invmassp @ emb_p.T

Anaogous, combining `Embedding` and the inverse mass matrix operation for `fes_u` allows to realize the following block inverse operations on `fes`:

$$
\underbrace{\left( \begin{array}{cc} 0 & 0 \\ 0 & M_{\mathbf{u}}^{-1} \end{array} \right)}_{\texttt{invu}} = 
\underbrace{\left( \begin{array}{c} I \\ 0 \end{array} \right)}_{\texttt{emb_u}} \cdot \underbrace{M_{\mathbf{u}}^{-1}}_{\texttt{invmassu}} \cdot \underbrace{\left( \begin{array}{cc} I & 0 \end{array} \right)}_{\texttt{emb_u.T}}
$$

In [ ]:
udofs = fes.Range(1)
emb_u = Embedding(fes.ndof, udofs)
invmassu = fes_u.Mass(Id(mesh.dim)).Inverse()
invu = emb_u @ invmassu @ emb_u.T

### Time loop 
Assuming the operators `B` and `BT` corresponding to $b_h(\cdot,\cdot)$ are given:

In [ ]:
# For the time stepping
def Run(B, BT, t0 = 0, tend = 2, tau = 2e-3, backward = False):
    t = 0
    with TaskManager():
        while t < (tend-t0) - tau/2:
            t += tau
            if not backward:
                gfu.vec.data += -tau * invp @ BT * gfu.vec
                gfu.vec.data += tau * invu @ B * gfu.vec
                print("\r t = {:}".format(t0 + t),end="")
            else:
                gfu.vec.data += -tau * invu @ B * gfu.vec
                gfu.vec.data += tau * invp @ BT * gfu.vec
                print("\r t = {:}".format(tend - t),end="")
            Redraw(blocking=False)
        print("")

Initial values:

In [ ]:
gfu.components[0].Set (exp(-100*((x-0.5)**2+y**2))+exp(-100*((x+0.5)**2+y**2)))
gfu.components[1].vec[:] = 0
Redraw()

## Version 1: The bilinear form for application on the full space `fes`

In [ ]:
n = specialcf.normal(mesh.dim) 
(p,u),(q,v) = fes.TnT()
B = BilinearForm(fes, nonassemble=True)
B += grad(p)*v * dx + 0.5*(p.Other()-p)*(v*n) * dx(element_boundary=True)
BT = BilinearForm(fes, nonassemble=True)
BT += grad(q)*u * dx + 0.5*(q.Other()-q)*(u*n) * dx(element_boundary=True)

In [ ]:
%%time 
Run(B.mat, BT.mat, backward=False)
Run(B.mat, BT.mat, backward=True)

## Version 1: The bilinear form for application on the full space `fes`

In [ ]:
B = BilinearForm(fes, nonassemble=True)
B += grad(p)*v * dx +  0.5*(p.Other()-p)*(v+v.Other())*n * dx(skeleton=True) - p * v*n * ds(skeleton=True)
BT = BilinearForm(fes, nonassemble=True)
BT += grad(q)*u * dx +  0.5*(q.Other()-q)*(u+u.Other())*n * dx(skeleton=True) - q * u*n * ds(skeleton=True)

In [ ]:
%%time 
Run(B.mat, BT.mat, backward=False)
Run(B.mat, BT.mat, backward=True)

In [ ]:
#gfu.components[0].Set (exp(-100*(x**2+y**2)))
#gfu.components[1].vec[:] = 0
#Redraw()

Using TraceOps:

In [ ]:
order = 1
fes_p = L2(mesh, order=order, all_dofs_together=True, order_policy=ORDER_POLICY.CONSTANT)

fes_tr = FacetFESpace(mesh, order=order)

traceop = fes_p.TraceOperator(fes_tr, False)


In [ ]:
f1 = FacetFESpace(mesh,order=0)
f2 = L2(mesh,order=0,all_dofs_together=True, order_policy=ORDER_POLICY.CONSTANT)

In [ ]:
traceop12 = f2.TraceOperator(f1, False)

In [ ]:
gf1 = GridFunction(f1)
gf2 = GridFunction(f2)

In [ ]:
gf2.Set(1)
print(gf2.vec)

In [ ]:
gf1.vec.data = traceop12 * gf2.vec
print(gf1.vec)

In [ ]:
fes_tr = FacetFESpace(mesh, order=order+1)

p = fes_p.TrialFunction()
v = fes_u.TestFunction()
phat = fes_tr.TrialFunction()

traceop = fes_p.TraceOperator(fes_tr, False)


  
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True) + 0.5 * p * (v*n) *ds(skeleton=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u)
Btr += 0.5 * phat * (v*n) * dx(element_boundary=True) 
Btr.Assemble()

B = emb_u @ (Bel.mat + Btr.mat @ traceop) @ emb_p.T
%time Run(B, B.T, backward=False)


In [ ]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += 0.5 * phat * (v*n) *dx(element_boundary=True)
Btr.Assemble()

traceop = fes_p.TraceOperator(fes_tr, False)
fullB = emb_u @ (Bel.mat + Btr.mat @ traceop) @ emb_p.T

Run(fullB, fullB.T, backward=True)
